<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pygame初期設定" data-toc-modified-id="Pygame初期設定-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pygame初期設定</a></span></li><li><span><a href="#テキストボックス" data-toc-modified-id="テキストボックス-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>テキストボックス</a></span></li><li><span><a href="#ボタン基底class" data-toc-modified-id="ボタン基底class-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>ボタン基底class</a></span></li><li><span><a href="#Makeボタンclass" data-toc-modified-id="Makeボタンclass-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Makeボタンclass</a></span></li><li><span><a href="#Randomボタンclass" data-toc-modified-id="Randomボタンclass-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Randomボタンclass</a></span></li><li><span><a href="#Foldボタンclass" data-toc-modified-id="Foldボタンclass-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Foldボタンclass</a></span></li><li><span><a href="#折り線class" data-toc-modified-id="折り線class-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>折り線class</a></span></li><li><span><a href="#地図class" data-toc-modified-id="地図class-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>地図class</a></span></li><li><span><a href="#main関数" data-toc-modified-id="main関数-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>main関数</a></span></li></ul></div>

# Pygame初期設定

In [1]:
import pygame
from pygame.locals import *
import sys
import random
from Sugar_map_folding import fold_check
# 平坦折り関数 fold_check(news_list)
# 戻り値: ['Error'], ['unfoldable'], foldable 解のリスト
from Sugar_map_folding import random_news_list
# M×N地図　ランダム入力 random_news_list(M, N)
# 戻り値: newsリスト

# Pygameの初期化
pygame.init()

# 画面の大きさ
SCREEN_SIZE = (800, 600)
screen = pygame.display.set_mode(SCREEN_SIZE)

# 地図の描画位置
MAP_START_Y = 200

# タイトルバー
pygame.display.set_caption("Map Folding")

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


# テキストボックス

In [2]:
# テキストボックス設定
COLOR_INACTIVE = pygame.Color('lightskyblue3')
COLOR_ACTIVE = pygame.Color('dodgerblue2')
FONT = pygame.font.Font(None, 32)

# テキストボックスclass
class InputBox:
    def __init__(self, x, y, w, h, text=''):
        self.rect = pygame.Rect(x, y, w, h)
        self.color = COLOR_INACTIVE
        self.text = text
        self.txt_surface = FONT.render(text, True, self.color)
        self.active = False
    
    # イベント処理メソッド
    def handle_event(self, event):
        
        # クリック時の操作
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos): # テキストボックス内をクリック
                self.active = not self.active
            else:
                self.active = False
        self.color = COLOR_ACTIVE if self.active else COLOR_INACTIVE
        
        # 入力時の処理
        if event.type == pygame.KEYDOWN:
            if self.active:
                if event.key == pygame.K_RETURN:
                    self.active = not self.active
                elif event.key == pygame.K_BACKSPACE:
                    self.text = self.text[:-1]
                elif event.key == pygame.K_TAB:
                    pass
                elif event.key == pygame.K_DELETE:
                    pass
                elif event.unicode.isdecimal():
                    self.text += event.unicode
                    # テキストSurfaceの作成を行い, 枠をはみ出せば削除
                    self.txt_surface = FONT.render(self.text, True, self.color)
                    if self.txt_surface.get_width()+5 > self.rect.w:
                        self.text = self.text[:-1]
                        
        # テキストSurfaceの作成
        self.txt_surface = FONT.render(self.text, True, self.color)
        
        
    # テキストボックスの描画メソッド
    def draw(self, screen):
        # テキストの描画
        txt_position = self.rect.x + self.rect.w - self.txt_surface.get_width() -5
        screen.blit(self.txt_surface, (txt_position, self.rect.y+5))
        # 長方形の描画
        pygame.draw.rect(screen, self.color, self.rect, 2)

# ボタン基底class

In [3]:
# ボタンの設定
# COLOR_INACTIVE = pygame.Color('lightskyblue3')
# COLOR_ACTIVE = pygame.Color('dodgerblue2')
# FONT = pygame.font.Font(None, 32)
        
        
# ボタンclass
class Button:
    def __init__(self, x, y, text=''):
        self.text = text
        self.color = COLOR_INACTIVE
        self.txt_surface = FONT.render(text, True, self.color)
        w, h = self.txt_surface.get_size()
        self.rect = pygame.Rect(x, y, w, h)
        self.active = False
        self.pushed = False
    
    # イベント処理メソッド
    def handle_event(self, event):
        # クリック時の操作
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.active:
                if self.rect.collidepoint(event.pos):
                    self.pushed = True
                    # print("clicked.")
        
    # ボタン描画メソッド
    def draw(self, screen):
        # テキストの描画
        if self.active:
            screen.blit(self.txt_surface, (self.rect.x, self.rect.y))
        
    # ボタンの更新
    def update(self):
        self.active = True
        
        # マウスが乗っている
        if self.rect.collidepoint(pygame.mouse.get_pos()):
            self.color = COLOR_ACTIVE
        else:
            self.color = COLOR_INACTIVE
        self.txt_surface = FONT.render(self.text, True, self.color)

# Makeボタンclass

In [4]:
# Makeボタンclass
class MakeButton(Button):
        
    # ボタンの更新
    def update(self, text1, text2):
        if text1.isdecimal() and text2.isdecimal():
            if int(text1) != 0 and int(text2) != 0:
                self.active = True
        else:
            self.active = False
        
        # マウスが乗っている
        if self.rect.collidepoint(pygame.mouse.get_pos()):
            self.color = COLOR_ACTIVE
        else:
            self.color = COLOR_INACTIVE
        self.txt_surface = FONT.render(self.text, True, self.color)

# Randomボタンclass

In [ ]:
# Randomボタンclass
class RandomButton(Button):
       
    # イベント処理メソッド
    def handle_event(self, event, map_obj):
        # クリック時の操作
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.active:
                if self.rect.collidepoint(event.pos):
                    map_obj.random_set()
                    
    # ボタンの更新
    def update(self, map_obj):
        if map_obj.m > 1 and map_obj.n > 1:
            self.active = True
        else:
            self.active = False
        
        # マウスが乗っている
        if self.rect.collidepoint(pygame.mouse.get_pos()):
            self.color = COLOR_ACTIVE
        else:
            self.color = COLOR_INACTIVE
        self.txt_surface = FONT.render(self.text, True, self.color)

# Foldボタンclass

In [ ]:
# Foldボタンclass
class FoldButton(Button):
    def __init__(self, x, y, text=''):
        self.text = text
        self.color = COLOR_INACTIVE
        self.txt_surface = FONT.render(text, True, self.color)
        w, h = self.txt_surface.get_size()
        self.rect = pygame.Rect(x, y, w, h)
        self.active = False
        self.output_surface = FONT.render('', True, pygame.Color('lightskyblue3'))
        self.output_active = False
       
    # イベント処理メソッド
    def handle_event(self, event, screen, map_obj):
        # クリック時の操作
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.active:
                if self.rect.collidepoint(event.pos):
                    # 計算中の表示
                    run_surface = FONT.render("RUNNING...", True, pygame.Color('darkorange'))
                    # 背景色で塗りつぶす
                    w, h = self.txt_surface.get_size()
                    black_rect = Rect(self.rect.x, self.rect.y, w, h)
                    pygame.draw.rect(screen, (30,30,30), black_rect)
                    screen.blit(run_surface, (self.rect.x, self.rect.y))
                    # 画面を更新
                    pygame.display.update()
                    
                    # 折り判定
                    if map_obj.m > 1 and map_obj.n > 1:
                        map_obj.fold()
                    else:
                        map_obj.answer = ['0']

                    
                    
    # ボタン描画メソッド
    def draw(self, screen):
        # ボタンの描画
        if self.active:
            screen.blit(self.txt_surface, (self.rect.x, self.rect.y))
        # 判定結果の描画
        if self.output_active:
            screen.blit(self.output_surface, (self.rect.x, self.rect.y))
        
        
    # ボタンの更新
    def update(self, map_obj):
        if map_obj.m > 1 and map_obj.n > 1:
            for directions in map_obj.news_list:
                if 'u' in directions:
                    self.active = False
                    break
                else:
                    self.active = True
        elif map_obj.m != 0 and map_obj.n != 0:
            self.active = True
        else:
            self.active = False
                
        # マウスが乗っている
        if self.rect.collidepoint(pygame.mouse.get_pos()):
            self.color = COLOR_ACTIVE
        else:
            self.color = COLOR_INACTIVE
        self.txt_surface = FONT.render(self.text, True, self.color)
        
        
        # 判定結果
        if len(map_obj.answer) == 0:
            self.output_active = False
        elif map_obj.answer[0].isdecimal(): # 平坦折り可能
            self.output_surface = FONT.render("foldable!", True, pygame.Color('dodgerblue2'))
            self.active = False
            self.output_active = True
        else:
            self.output_surface = FONT.render(map_obj.answer[0], True, pygame.Color('lightskyblue3'))
            self.active = False
            self.output_active = True

# 折り線class

In [ ]:
# 折り線の設定
COLOR_MOUNTAIN = pygame.Color('red')
COLOR_VALLEY = pygame.Color('deepskyblue')
THICK = 3

# 水平折り線class
class h_crease:
    def __init__(self, x, y, length, TOLERANCE):
        self.rect = pygame.Rect(x, y-THICK/2, length, THICK)
        self.CollRect = pygame.Rect(x, y-TOLERANCE, length, TOLERANCE*2)
        self.color = COLOR_VALLEY
        self.mountain = False
        
    # イベント処理メソッド
    def handle_event(self, event):
        crease_changed = False
        # クリック時の操作
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.CollRect.collidepoint(event.pos): # 折り線をクリック
                self.mountain = not self.mountain
                crease_changed = True
        self.color = COLOR_MOUNTAIN if self.mountain else COLOR_VALLEY
        return crease_changed
        
    # テキストボックスの描画メソッド
    def draw(self, screen):
        # 長方形の描画
        pygame.draw.rect(screen, self.color, self.rect)
        
# 垂直折り線class
class v_crease(h_crease):
    def __init__(self, x, y, length, TOLERANCE):
        self.rect = pygame.Rect(x-THICK/2, y, THICK, length)
        self.CollRect = pygame.Rect(x-TOLERANCE, y, TOLERANCE*2, length)
        self.color = COLOR_VALLEY
        self.mountain = False

# 地図class

In [ ]:
# 地図classの設定
# THICK = 3
               
# 地図class
class Map:
    def __init__(self, x, y, m, n, cell_size):
        self.size = cell_size
        self.rect = pygame.Rect(x, y, cell_size*n, cell_size*m)
        self.m = m
        self.n = n
        self.h_creases = []
        self.v_creases = []
        self.news_list = []
        self.answer = []
        
    # 折り線オブジェクトの作成
    def makeCreaseObject(self):
        # 水平折り線オブジェクト
        for i in range(1, self.m):
            for j in range(self.n):
                x = self.rect.x + self.size * j
                y = self.rect.y + self.size * i
                crease_obj = h_crease(x, y, self.size, self.size/8)
                self.h_creases.append(crease_obj)
        # 垂直折り線オブジェクト
        for i in range(self.m):
            for j in range(1, self.n):
                x = self.rect.x + self.size * j
                y = self.rect.y + self.size * i
                crease_obj = v_crease(x, y, self.size, self.size/8)
                self.v_creases.append(crease_obj)
    
    
    # 地図の描画メソッド
    def draw(self, screen):        
        # 折り線の描画                
        for crease in self.h_creases + self.v_creases:
            crease.draw(screen)
            
        # 長方形の描画
        pygame.draw.rect(screen, (255, 255, 255), self.rect, THICK)
            
        # 前川定理checkerの描画
        self.make_news()
        for i in range(len(self.news_list)):
            for j in range(len(self.news_list[i])):
                if self.news_list[i][j] == 'u':
                    # 長方形の描画
                    x = self.rect.x + self.size * (j+1) - 5
                    y = self.rect.y + self.size * (i+1) - 5
                    pygame.draw.rect(screen, (255,255,0), Rect(x,y,10,10), 1)
         
    
    # NEWS系列の作成
    def make_news(self):
        m = self.m
        n = self.n
        # self.news_listを初期化
        self.news_list.clear()
        
        
        add_list = []
        for i in range(m-1):
            
            # add_listを初期化
            add_list.clear()
            
            for j in range(n-1):
                north = self.v_creases[(n-1)*i+j].mountain
                east = self.h_creases[n*i+j+1].mountain
                west = self.h_creases[n*i+j].mountain
                south = self.v_creases[(n-1)*(i+1)+j].mountain
                if (north+east+west+south) == 1: # 一つだけ山折り
                    if north:
                        add_list += 'n'
                    elif east:
                        add_list += 'e'
                    elif west:
                        add_list += 'w'
                    elif south:
                        add_list += 's'
                elif (north+east+west+south) == 3: # 一つだけ谷折り
                    if not north:
                        add_list += 'n'
                    elif not east:
                        add_list += 'e'
                    elif not west:
                        add_list += 'w'
                    elif not south:
                        add_list += 's'
                else:
                    add_list += 'u'
            self.news_list.append(add_list[:])
            
            
    # newsリストに従って折り線オブジェクトをセット
    # 引数 : 左端の折り線が山折り->True
    def news_set(self, left_mountain):
        m = self.m
        n = self.n
        # 左端の折り線を谷で固定
        self.h_creases[0].mountain = left_mountain
        # 一行目
        for j in range(n-1):
            if self.news_list[0][j] == 'n':
                self.v_creases[j].mountain = not self.h_creases[j].mountain
                self.h_creases[j+1].mountain = self.h_creases[j].mountain
                self.v_creases[n-1+j].mountain = self.h_creases[j].mountain
            elif self.news_list[0][j] == 'e':
                self.v_creases[j].mountain = self.h_creases[j].mountain
                self.h_creases[j+1].mountain = not self.h_creases[j].mountain
                self.v_creases[n-1+j].mountain = self.h_creases[j].mountain
            elif self.news_list[0][j] == 'w':
                self.v_creases[j].mountain = not self.h_creases[j].mountain
                self.h_creases[j+1].mountain = not self.h_creases[j].mountain
                self.v_creases[n-1+j].mountain = not self.h_creases[j].mountain
            elif self.news_list[0][j] == 's':
                self.v_creases[j].mountain = self.h_creases[j].mountain
                self.h_creases[j+1].mountain = self.h_creases[j].mountain
                self.v_creases[n-1+j].mountain = not self.h_creases[j].mountain
                
        # 二行目以降                
        for i in range(1, m-1):
            for j in range(n-1):
                if self.news_list[i][j] == 'n':
                    self.h_creases[n*i+j].mountain = not self.v_creases[(n-1)*i+j].mountain
                    self.v_creases[(n-1)*(i+1)+j].mountain = not self.v_creases[(n-1)*i+j].mountain
                    self.h_creases[n*i+j+1].mountain = not self.v_creases[(n-1)*i+j].mountain
                elif self.news_list[i][j] == 'e':
                    self.h_creases[n*i+j].mountain = self.v_creases[(n-1)*i+j].mountain
                    self.v_creases[(n-1)*(i+1)+j].mountain = self.v_creases[(n-1)*i+j].mountain
                    self.h_creases[n*i+j+1].mountain = not self.v_creases[(n-1)*i+j].mountain
                elif self.news_list[i][j] == 'w':
                    self.h_creases[n*i+j].mountain = not self.v_creases[(n-1)*i+j].mountain
                    self.v_creases[(n-1)*(i+1)+j].mountain = self.v_creases[(n-1)*i+j].mountain
                    self.h_creases[n*i+j+1].mountain = self.v_creases[(n-1)*i+j].mountain
                elif self.news_list[i][j] == 's':
                    self.h_creases[n*i+j].mountain = self.v_creases[(n-1)*i+j].mountain
                    self.v_creases[(n-1)*(i+1)+j].mountain = not self.v_creases[(n-1)*i+j].mountain
                    self.h_creases[n*i+j+1].mountain = self.v_creases[(n-1)*i+j].mountain
                

    # ランダムセット
    def random_set(self):
        # M×N地図　ランダム入力 random_news_list(M, N)
        # 戻り値: newsリスト
        self.news_list = random_news_list(self.m, self.n)
        left_mountain = random.choice([True, False])
        self.news_set(left_mountain)
        self.answer.clear()
        

    
    # 折り判定
    def fold(self):
        # fold_check(news_list)
        # 戻り値: ['Error'], ['unfoldable'], foldable 解のリスト
        self.answer = fold_check(self.news_list)

# main関数

In [ ]:
def main():
    clock = pygame.time.Clock()
    m_input = InputBox(100, 100, 36, 32)
    n_input = InputBox(166, 100, 36, 32)
    input_boxes = [m_input, n_input]
    makeButton = MakeButton(388, 105, "Make")
    map_was_drawn = False
    randomButton = RandomButton(484, 105, "Random")
    foldButton = FoldButton(612, 105, "Fold")
    
    
    Running = True
    while Running:
        
        # 背景色の設定
        screen.fill((30, 30, 30))
        
        # テキストボックスの描画
        for box in input_boxes:
            box.draw(screen)
        # タイトルの描画
        title_surface1 = FONT.render("×", True, pygame.Color('lightskyblue3'))
        screen.blit(title_surface1, (146, 100))
        title_surface2 = FONT.render("Map", True, pygame.Color('lightskyblue3'))
        screen.blit(title_surface2, (238, 105))
        
        # ボタンの描画
        makeButton.draw(screen)
        randomButton.draw(screen)
        foldButton.draw(screen)
        
        
        if makeButton.pushed:
            # CELL_SIZEの決定
            m = int(m_input.text)
            n = int(n_input.text)
            x_size = SCREEN_SIZE[0] / (n + 4)
            y_size = (SCREEN_SIZE[1] - MAP_START_Y) / (m + 1)
            CELL_SIZE = min(x_size, y_size)
            MAP_START_X = (SCREEN_SIZE[0] - CELL_SIZE * n) / 2

            # 地図オブジェクトの作成
            map_obj = Map(MAP_START_X, MAP_START_Y, m, n, CELL_SIZE)
            map_obj.makeCreaseObject()
            map_was_drawn = True
            makeButton.pushed = False
        
        # 地図の描画
        if map_was_drawn:
            map_obj.make_news()
            map_obj.draw(screen)
            
        
            
        # 画面を更新
        pygame.display.update()
        clock.tick(30)
        # イベント処理
        for event in pygame.event.get():
            
            # 閉じるボタンが押されたら終了
            if event.type == pygame.QUIT:
                Running = False
            
            # テキストボックスのイベント処理
            for box in input_boxes:
                box.handle_event(event)
                
            # ボタンのイベント処理
            makeButton.handle_event(event)
            makeButton.update(m_input.text, n_input.text)
            if map_was_drawn:
                foldButton.handle_event(event, screen, map_obj)
                foldButton.update(map_obj)
                randomButton.handle_event(event, map_obj)
                randomButton.update(map_obj)
            
            
            #　折り線のイベント処理
            if map_was_drawn:
                crease_change = 0
                for crease in map_obj.h_creases + map_obj.v_creases:
                    crease_change += crease.handle_event(event)
                    
                if crease_change > 0: # 折り線が変更されれば
                    map_obj.answer.clear()
                    
            


if __name__ == '__main__':
    main()
    pygame.quit()

5 × 5 地図折り問題  [['w', 'e', 's', 's'], ['s', 's', 's', 's'], ['w', 'e', 'n', 'w'], ['e', 'n', 'e', 's']]
s SATISFIABLE
a m_1	9
a m_2	7
a m_3	4
a m_4	21
a m_5	25
a m_6	23
a m_7	2
a m_8	3
a m_9	22
a m_10	24
a m_11	19
a m_12	18
a m_13	17
a m_14	20
a m_15	1
a m_16	11
a m_17	12
a m_18	13
a m_19	10
a m_20	8
a m_21	16
a m_22	15
a m_23	14
a m_24	5
a m_25	6
a

3 × 3 地図折り問題  [['s', 'e'], ['e', 's']]
s SATISFIABLE
a m_1	9
a m_2	8
a m_3	5
a m_4	1
a m_5	7
a m_6	6
a m_7	2
a m_8	3
a m_9	4
a

